# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Reading CSV file
gmaps.configure(api_key=g_key)
city_weather_df = pd.read_csv("../output_data/cities.csv")
city_weather_df


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Ponta do Sol,32.6667,-17.1000,69.80,60,20,6.91,PT,1620471614
1,Rikitea,-23.1203,-134.9692,78.12,78,17,10.20,PF,1620471614
2,Qaanaaq,77.4840,-69.3632,22.77,93,100,2.37,GL,1620471615
3,San Cristobal,7.7669,-72.2250,65.23,96,100,2.28,VE,1620471615
4,Rome,43.2128,-75.4557,43.00,87,90,7.18,US,1620471615
...,...,...,...,...,...,...,...,...,...
558,Aitape,-3.1374,142.3475,80.91,84,75,4.23,PG,1620471783
559,Caravelas,-17.7125,-39.2481,75.70,86,13,7.67,BR,1620471784
560,Pedra Branca,-5.4542,-39.7172,71.19,95,64,3.98,BR,1620471653
561,Santa Inês,-3.6667,-45.3800,77.00,91,82,2.86,BR,1620471538


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Read GKEY
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Humidity DF
humidity = city_weather_df["Humidity"]

In [6]:
#configure MAP
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Create new Data Frame, and filter
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
94,Cabo San Lucas,22.8909,-109.9124,75.0,42,0,9.51,MX,1620471556
312,Baiyin,36.5583,104.2081,78.8,8,0,4.47,CN,1620471710
437,Casablanca,33.5928,-7.6192,78.8,68,0,9.22,MA,1620471749


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Create Hotel DF
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
94,Cabo San Lucas,MX,22.8909,-109.9124,
312,Baiyin,CN,36.5583,104.2081,
437,Casablanca,MA,33.5928,-7.6192,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [12]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 94: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Bahia Hotel & Beach House.
--------------------Ending Search
Retrieving Results for Index 312: Baiyin.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 437: Casablanca.
Closest hotel in Casablanca is Four Seasons Hotel Casablanca.
--------------------Ending Search


In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
94,Cabo San Lucas,MX,22.8909,-109.9124,Bahia Hotel & Beach House
312,Baiyin,CN,36.5583,104.2081,
437,Casablanca,MA,33.5928,-7.6192,Four Seasons Hotel Casablanca


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))